# Object Detection Pipeline - Hyperparameter Tuning & Model Training

Hyperparameter tuning with Ray Tune on validation set, followed by final training on train+val and evaluation on test set.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kshitijrajsharma/dl4cv-oda/blob/master/notebooks/experiment.ipynb)

In [1]:
# ! pip install dl4cv_oda ray[tune]

In [2]:
import os
import yaml
import shutil
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from dl4cv_oda import (clean_osm_data, clip_labels_to_tiles, convert_to_yolo_format,
                       create_train_val_split, create_yolo_config, download_tiles)
from ultralytics import YOLO, RTDETR

## Configuration

In [3]:
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
CHIPS_DIR = DATA_DIR / "chips"
LABELS_DIR = DATA_DIR / "labels"
YOLO_DIR = DATA_DIR / "yolo"

OSM_FILE = RAW_DIR / "kolovai-trees.geojson"
CLEANED_FILE = RAW_DIR / "cleaned.geojson"
TREES_BOX_FILE = DATA_DIR / "trees_box.geojson"
TILES_FILE = DATA_DIR / "tiles.geojson"

IMG_SIZE = 256
EPOCHS = 50
PATIENCE = 10

TUNE_EPOCHS = 10
TUNE_ITERATIONS = 2
GPU_PER_TRIAL = 1

MODELS_TO_TRAIN = {
    "yolov8l": "yolov8l.pt",
    "yolo12l": "yolo12l.pt",
    "rtdetr-l": "rtdetr-l.pt"
}

In [4]:
import requests
if not OSM_FILE.exists():
    OSM_FILE.parent.mkdir(parents=True, exist_ok=True)
    OSM_FILE.write_bytes(requests.get("https://github.com/kshitijrajsharma/dl4cv-oda/blob/master/data/raw/kolovai-trees.geojson?raw=true", allow_redirects=True).content)
    print(f"Downloaded {OSM_FILE}")
else:
    print("OSM data ready")

OSM data ready


## Step 1: Clean OSM Data

In [5]:
if not CLEANED_FILE.exists():
    count = clean_osm_data(str(OSM_FILE), str(CLEANED_FILE), str(TREES_BOX_FILE))
    print(f"Processed {count} trees")
else:
    print("Cleaned data ready")

Cleaned data ready


## Download Tiles

In [6]:
if not TILES_FILE.exists():
    data = gpd.read_file(TREES_BOX_FILE)
    data.to_crs(epsg=4326, inplace=True)
    bbox = list(data.total_bounds)
    await download_tiles(bbox, 19, "https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/{z}/{x}/{y}.png", DATA_DIR, 'OAM')
    print("Tiles downloaded")

## Clip Labels & Convert to YOLO Format

In [7]:
if TILES_FILE.exists() and not (YOLO_DIR / "train").exists():
    stats = clip_labels_to_tiles(str(TREES_BOX_FILE), str(TILES_FILE), str(LABELS_DIR))
    print(f"{stats['processed']} tiles, {stats['total_trees']} trees")
    class_mapping = convert_to_yolo_format(str(TREES_BOX_FILE), str(CHIPS_DIR), str(LABELS_DIR), str(YOLO_DIR), target_species="Coconut")
    print(f"Class mapping: {class_mapping}")
else:
    print("YOLO format data ready")

YOLO format data ready


## Create Train/Val/Test Split

In [8]:
if not (YOLO_DIR / "train").exists():
    train_count, val_count, test_count = create_train_val_split(str(LABELS_DIR), str(CHIPS_DIR), str(YOLO_DIR), train_ratio=0.7, val_ratio=0.2, test_ratio=0.1)
    print(f"Train: {train_count} | Val: {val_count} | Test: {test_count}")
else:
    print("Train/Val/Test split ready")

config_file = create_yolo_config(str(YOLO_DIR), {"Coconut": 0})
config_file_abs = os.path.abspath(config_file)
print(f"Config: {config_file_abs}")

Train/Val/Test split ready
Config: /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/data/yolo/config.yaml


## Hyperparameter Tuning with Ray Tune

Tune all models with consistent parameters.

In [9]:
tune_results = {}
for model_name, model_weights in MODELS_TO_TRAIN.items():
    print(f"Tuning {model_name}...")
    model = RTDETR(model_weights) if "rtdetr" in model_name.lower() else YOLO(model_weights)
    result_grid = model.tune(data=config_file_abs, epochs=TUNE_EPOCHS, imgsz=IMG_SIZE, 
                            patience=PATIENCE, iterations=TUNE_ITERATIONS, 
                            gpu_per_trial=GPU_PER_TRIAL, use_ray=True)
    tune_results[model_name] = result_grid
    print(f"{model_name} tuning complete")

(_tune pid=3363774) New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
(_tune pid=3363774) Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
(_tune pid=3363774) engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.17653029184586422, box=0.12676926390294507, cache=False, cfg=None, classes=None, close_mosaic=10, cls=2.1328266658032673, compile=False, conf=None, copy_paste=0.8968784262700898, copy_paste_mode=flip, cos_lr=False, cutmix=0.15563666390661335, data=/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/data/yolo/config.yaml, degrees=24.541441086111952, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.677922989377532, flipud=0.05200689054963137, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0946117969553

(_tune pid=3363774) /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
(_tune pid=3363774)   return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       1/10      4.54G      1.666      0.864     0.5373        374        256: 0% ──────────── 0/28  2.2s
       1/10      4.54G      1.629     0.9936     0.5695        291        256: 4% ──────────── 1/28 1.5it/s 2.4s<18.4s
       1/10      4.54G      1.629     0.9936     0.5695        291        256: 4% ──────────── 1/28 1.5it/s 2.4s<18.4s
       1/10      4.54G      1.617      1.185     0.5627        193        256: 7% ╸─────────── 2/28 2.6it/s 2.6s<9.9s
       1/10      4.54G      1.617      1.185     0.5627        193        256: 7% ╸─────────── 2/28 2.6it/s 2.6s<9.9s
       1/10      4.54G      1.661      1.099     0.5618        376        256: 11% ━─────────── 3/28 3.5it/s 2.8s<7.2s
       1/10      4.54G      1.661      1.099     0.5618        376        256: 11% ━─────────── 3/28 3.5it/s 2.8s<7.2s
       1/10      4.65G      1.676      1.016     0.5578        503        256: 14% ━╸────────── 4/28 4.0it/s 2.9s<6.0s
       1/10      4.65G      1.676      1.016     0.5578        

(_tune pid=3363774) /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/ray/train/_internal/session.py:792: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(_tune pid=3363774)   _log_deprecation_warning(
(_tune pid=3363774) /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
(_tune pid=3363774)   return Variable

       2/10      6.47G      1.403     0.5368     0.4517        277        256: 0% ──────────── 0/28  0.2s
       2/10      6.47G      1.313     0.5045     0.3786        408        256: 4% ──────────── 1/28 1.8it/s 0.3s<14.9s
       2/10      6.47G      1.256     0.5253     0.3438        258        256: 7% ╸─────────── 2/28 3.2it/s 0.5s<8.2s
       2/10      6.47G      1.212     0.5288     0.3403        399        256: 11% ━─────────── 3/28 4.3it/s 0.6s<5.8s
       2/10      6.47G      1.256     0.5253     0.3438        258        256: 7% ╸─────────── 2/28 3.2it/s 0.5s<8.2s
       2/10      6.47G      1.212     0.5288     0.3403        399        256: 11% ━─────────── 3/28 4.3it/s 0.6s<5.8s
       2/10      6.47G      1.164     0.5505     0.3196        300        256: 14% ━╸────────── 4/28 4.8it/s 0.8s<5.0s
       2/10      6.47G      1.163     0.5518     0.3175        385        256: 18% ━━────────── 5/28 5.5it/s 0.9s<4.2s
       2/10      6.47G      1.164     0.5505     0.3196        

...

       9/10         7G     0.6156     0.5568      0.146        340        256: 4% ──────────── 1/28 2.1it/s 0.3s<12.7s
       9/10         7G     0.6234     0.5605     0.1486        476        256: 7% ╸─────────── 2/28 3.5it/s 0.4s<7.4s
       9/10         7G     0.6481     0.5647     0.1563        351        256: 11% ━─────────── 3/28 4.3it/s 0.6s<5.8s
       9/10         7G     0.6404     0.5612     0.1549        318        256: 14% ━╸────────── 4/28 5.0it/s 0.7s<4.8s
       9/10         7G     0.6481     0.5647     0.1563        351        256: 11% ━─────────── 3/28 4.3it/s 0.6s<5.8s
       9/10         7G     0.6404     0.5612     0.1549        318        256: 14% ━╸────────── 4/28 5.0it/s 0.7s<4.8s
       9/10         7G     0.6419     0.5558     0.1513        348        256: 18% ━━────────── 5/28 5.5it/s 0.9s<4.1s
       9/10         7G     0.6419     0.5558     0.1513        348        256: 18% ━━────────── 5/28 5.5it/s 0.9s<4.1s
       9/10         7G     0.6555     0.5528     

(_tune pid=3369759) /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/ray/train/_internal/session.py:792: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(_tune pid=3369759)   _log_deprecation_warning(
(_tune pid=3369759) /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
(_tune pid=3369759)   return Variable

      10/10         7G     0.6251     0.5245     0.1428        422        256: 4% ──────────── 1/28 1.9it/s 0.3s<14.4s
      10/10         7G     0.6358     0.5444     0.1457        331        256: 7% ╸─────────── 2/28 3.2it/s 0.5s<8.0s
      10/10         7G     0.6295     0.5335     0.1459        340        256: 11% ━─────────── 3/28 4.1it/s 0.6s<6.2s
      10/10         7G     0.6195     0.5356     0.1407        242        256: 14% ━╸────────── 4/28 4.8it/s 0.8s<5.0s
      10/10         7G     0.6295     0.5335     0.1459        340        256: 11% ━─────────── 3/28 4.1it/s 0.6s<6.2s
      10/10         7G     0.6195     0.5356     0.1407        242        256: 14% ━╸────────── 4/28 4.8it/s 0.8s<5.0s
      10/10         7G       0.61     0.5349     0.1408        395        256: 18% ━━────────── 5/28 5.5it/s 0.9s<4.2s
      10/10         7G      0.611     0.5426     0.1385        277        256: 21% ━━╸───────── 6/28 5.9it/s 1.1s<3.8s
      10/10         7G       0.61     0.5349     

2026-01-11 00:49:02,545	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/notebooks/runs/detect/tune13' in 0.0017s.
2026-01-11 00:49:02,548	INFO tune.py:1041 -- Total run time: 133.01 seconds (133.00 seconds for the tuning loop).
2026-01-11 00:49:02,548	INFO tune.py:1041 -- Total run time: 133.01 seconds (133.00 seconds for the tuning loop).


rtdetr-l tuning complete


## Analyze Results & Extract Best Configs

In [10]:
def analyze_and_save_results(result_grid, model_name):
    results_list = []
    for i, result in enumerate(result_grid):
        df = result.metrics_dataframe
        p = df['metrics/precision(B)'].iloc[-1] if 'metrics/precision(B)' in df else 0
        r = df['metrics/recall(B)'].iloc[-1] if 'metrics/recall(B)' in df else 0
        m = df['metrics/mAP50(B)'].iloc[-1] if 'metrics/mAP50(B)' in df else 0
        f1 = 2 * (p * r) / (p + r + 1e-6) if (p + r) > 0 else 0
        results_list.append({'trial': i, 'config': result.config, 'precision': p, 'recall': r, 'mAP50': m, 'f1': f1})
    
    results_df = pd.DataFrame(results_list).sort_values('f1', ascending=False)
    best = results_df.iloc[0]
    best_config = best['config']
    
    with open(YOLO_DIR / f"best_config_{model_name}.yaml", 'w') as f:
        yaml.dump(best_config, f, default_flow_style=False)
    
    print(f"{model_name:12s} F1: {best['f1']:.4f} | P: {best['precision']:.4f} | R: {best['recall']:.4f} | mAP50: {best['mAP50']:.4f}")
    return results_df, best_config

best_configs = {}
for model_name, result_grid in tune_results.items():
    _, best_configs[model_name] = analyze_and_save_results(result_grid, model_name)

yolov8l      F1: 0.4393 | P: 0.4415 | R: 0.4370 | mAP50: 0.3069
yolo12l      F1: 0.6483 | P: 0.6561 | R: 0.6406 | mAP50: 0.5692
rtdetr-l     F1: 0.7386 | P: 0.7624 | R: 0.7162 | mAP50: 0.6844


## Merge Train+Val for Final Training

In [11]:
trainval_dir = YOLO_DIR / "trainval"
trainval_config = YOLO_DIR / "data_trainval.yaml"

if not trainval_dir.exists():
    trainval_dir.mkdir(exist_ok=True)
    for split in ["train", "val"]:
        for f in (YOLO_DIR / split).glob("*"):
            shutil.copy(f, trainval_dir / f.name)
    with open(config_file, 'r') as f:
        trainval_config.write_text(f.read().replace("train: train", "train: trainval"))
    print(f"Merged train+val into {trainval_dir}")
else:
    print("Train+val merge ready")

Merged train+val into ../data/yolo/trainval


## Final Training with Best Hyperparameters

In [13]:
trained_models = {}
for model_name, model_weights in MODELS_TO_TRAIN.items():
    print(f"\nTraining {model_name} on train+val...")
    model = RTDETR(model_weights) if "rtdetr" in model_weights.lower() else YOLO(model_weights)
    cfg = best_configs[model_name].copy()
    cfg.update({'data': str(trainval_config), 'epochs': EPOCHS, 'imgsz': IMG_SIZE, 
                'patience': PATIENCE, 'name': f"final_{model_name}", 'plots': True})
    model.train(**cfg)
    trained_models[model_name] = model
    print(f"{model_name} complete")


Training yolov8l on train+val...
New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.7032215998455474, box=0.08238395960119319, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5396488654807599, compile=False, conf=None, copy_paste=0.04463147049762817, copy_paste_mode=flip, cos_lr=False, cutmix=0.6158661195794367, data=../data/yolo/data_trainval.yaml, degrees=7.891566853102933, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.6578521028477055, flipud=0.4593305214064205, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.05854238000775572, hsv_s=0.7327759186578207, hsv_v=0.06983565999105049, imgsz=256, int8=Fals

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       1/50      9.69G      1.697     0.2859     0.6311        571        256: 100% ━━━━━━━━━━━━ 29/29 2.8it/s 10.5s0.3s
       1/50      9.69G      1.697     0.2859     0.6311        571        256: 100% ━━━━━━━━━━━━ 29/29 2.8it/s 10.5s0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 4.3it/s 1.4s0.2s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 4.3it/s 1.4s
                   all        188       5167     0.0366      0.256     0.0251    0.00478
                   all        188       5167     0.0366      0.256     0.0251    0.00478

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
       2/50      8.92G      1.559     0.2732     0.6041       1451        256: 0% ──────────── 0/29  0.3s

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       2/50      8.97G       1.24     0.3987     0.3692        674        256: 100% ━━━━━━━━━━━━ 29/29 4.5it/s 6.4s0.2s

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 12.9it/s 0.5s.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 12.9it/s 0.5s
                   all        188       5167      0.466      0.449      0.393      0.122
                   all        188       5167      0.466      0.449      0.393      0.122

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
       3/50      7.19G      1.191     0.4077     0.2997       1034        256: 0% ──────────── 0/29  0.2s

...

      48/50      7.18G     0.6099     0.5345     0.1426        255        256: 100% ━━━━━━━━━━━━ 29/29 6.6it/s 4.4s0.1s
      48/50      7.18G     0.6099     0.5345     0.1426        255        256: 100% ━━━━━━━━━━━━ 29/29 6.6it/s 4.4s0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 14.8it/s 0.4s.3s
                   all        188       5167      0.469     0.0587     0.0617     0.0317
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 14.8it/s 0.4s
                   all        188       5167      0.469     0.0587     0.0617     0.0317

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      49/50      7.18G     0.6101     0.5305     0.1417        409        256: 3% ──────────── 1/29 2.0it/s 0.3s<13.7s

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      49/50      7.18G     0.6096     0.5347     0.1384        214        256: 100% ━━━━━━━━━━━━ 29/29 6.7it/s 4.3s0.1s
      49/50      7.18G     0.6096     0.5347     0.1384        214        256: 100% ━━━━━━━━━━━━ 29/29 6.7it/s 4.3s0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 14.0it/s 0.4s.3s
                   all        188       5167      0.675      0.168      0.177     0.0826
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 14.0it/s 0.4s
                   all        188       5167      0.675      0.168      0.177     0.0826

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      50/50      7.18G     0.6222     0.5282      0.159        365        256: 3% ──────────── 1/29 2.2it/s 0.3s<13.0s

/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:841: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:148.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      50/50      7.18G     0.6094     0.5363     0.1446        225        256: 100% ━━━━━━━━━━━━ 29/29 6.6it/s 4.4s0.1s
      50/50      7.18G     0.6094     0.5363     0.1446        225        256: 100% ━━━━━━━━━━━━ 29/29 6.6it/s 4.4s0.1s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 12.6it/s 0.5s.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 12.6it/s 0.5s
                   all        188       5167      0.735      0.494      0.477      0.174
                   all        188       5167      0.735      0.494      0.477      0.174

50 epochs completed in 0.099 hours.

50 epochs completed in 0.099 hours.
Optimizer stripped from /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/notebooks/runs/detect/final_rtdetr-l/weights/last.pt, 66.2MB
Optimizer stripped from /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/notebooks/runs/d

## Evaluate on Test Set

In [ ]:
test_results = {}
for name, model in trained_models.items():
    print(f"Evaluating {name}...")
    m = model.val(split='test')
    p, r = m.box.mp, m.box.mr
    test_results[name] = {"precision": p, "recall": r, "f1": 2 * (p * r) / (p + r + 1e-6),
                         "mAP50": m.box.map50, "mAP50-95": m.box.map}

Evaluating yolov8l...
Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
Model summary (fused): 112 layers, 43,607,379 parameters, 0 gradients, 164.8 GFLOPs
Model summary (fused): 112 layers, 43,607,379 parameters, 0 gradients, 164.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4935.2±1231.5 MB/s, size: 148.8 KB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4935.2±1231.5 MB/s, size: 148.8 KB)
val: Scanning /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/data/yolo/test... 45 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 45/45 3.1Kit/s 0.0s
val: /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/data/yolo/test/OAM-6783-293582-19.png: 1 duplicate labels removed
val: New cache created: /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/data/yolo/test.cache
val: Scanning /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/data/yolo/test... 45 images, 0 backgr

In [15]:
results_df = pd.DataFrame(test_results).T.sort_values('f1', ascending=False)
print("Test Set Results:")
print(results_df.to_string())
print(f"\nBest: {results_df.index[0]}")

Test Set Results:
          precision    recall        f1     mAP50  mAP50-95
rtdetr-l   0.687226  0.680886  0.684041  0.617940  0.262814
yolo12l    0.668094  0.652396  0.660151  0.548510  0.218083
yolov8l    0.648318  0.595311  0.620684  0.518881  0.194038

Best: rtdetr-l


## F1 Score per Epoch

In [16]:
n = len(MODELS_TO_TRAIN)
cols = min(3, n)
rows = (n + cols - 1) // cols
fig, axes = plt.subplots(rows, cols, figsize=(6*cols, 5*rows))
axes = axes.flatten() if n > 1 else [axes]

for idx, model_name in enumerate(MODELS_TO_TRAIN.keys()):
    csv = Path(f"runs/detect/final_{model_name}/results.csv")
    if csv.exists():
        df = pd.read_csv(csv)
        df.columns = df.columns.str.strip()
        p, r = df['metrics/precision(B)'], df['metrics/recall(B)']
        f1 = 2 * (p * r) / (p + r + 1e-6)
        axes[idx].plot(f1, label='F1', linewidth=2, color='#2E86AB')
        axes[idx].plot(p, label='Precision', alpha=0.7, color='#A23B72')
        axes[idx].plot(r, label='Recall', alpha=0.7, color='#F18F01')
        axes[idx].set_title(model_name.upper(), fontsize=11, fontweight='bold')
        axes[idx].set_xlabel('Epoch')
        axes[idx].set_ylabel('Score')
        axes[idx].legend()
        axes[idx].grid(True, alpha=0.3)

for idx in range(n, len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.savefig('f1_scores_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

<Figure size 1800x500 with 3 Axes>